# 层级索引

当目前为止，我们接触的都是一维数据和二维数据，用 `Pandas` 的 `Series` 和 `DataFrame` 对象就可以存储。  
但我们也经常会遇到存储多维数据的需求，数据索引超过一两个键。因此，`Pandas` 提供了 `Panel` 和 `Panel4D` 对象解决三维数据与四维数据。  
而在实践中，更直观的形式是通过**层级索引**（`hierarchical indexing`，也被称为**多级索引**，`multi-indexing`）配合多个有不同**等级**（`level`）的一级索引一起使用，  
这样就可以将高维数组转换成类似一维 `Series` 和二维 `DataFrame` 对象的形式。  

在这一节中，我们将介绍创建 `MultiIndex` 对象的方法，多级索引数据的取值、切片和统计值的计算，以及普通索引与层级索引的转换方法。

In [1]:
import numpy as np
import pandas as pd

## 1. 多级索引Series

让我们看看如何用一维的 `Series` 对象表示二维数据——用一系列包含特征与数值的数据点来简单演示。

### 1.1. 笨方法

In [2]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)  # 通过元组构成的多级索引
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]  # 假如你想要选择所有 2000 年的数据... ...

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### 1.2. 好方法：Pandas多级索引

`Pandas` 提供了更好的解决方案。用元组表示索引其实是多级索引的基础，`Pandas` 的 `MultiIndex` 类型提供了更丰富的操作方法。我们可以用元组创建一个多级索引。

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]  # 假如你想要选择所有 2000 年的数据 ^v^

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### 1.3. 高级数据的多级索引

你可能已经注意到，我们其实完全可以用一个带行列索引的简单 `DataFrame` 代替前面的多级索引。  
其实 `Pandas` 已经实现了类似的功能。`unstack()` 方法可以快速将一个多级索引的 `Series` 转化为普通索引的 `DataFrame`：

In [8]:
pop_df = pop.unstack()  # unstack() 方法“升维”数据
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()          # stack() 方法实现相反的效果，“降维”数据

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

你可能会纠结于为什么要费时间研究层级索引。其实理由很简单：  
如果我们可以用含多级索引的一维 `Series` 数据表示二维数据，那么我们就可以用 `Series` 或 `DataFrame` 表示三维甚至更高维度的数据。  
多级索引每增加一级，就表示数据增加一维，利用这一特点就可以轻松表示任意维度的数据了。

In [10]:
# 给DataFrame“升维”
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']  # 前面介绍过的通用函数和其他功能也同样适用于层级索引。
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [12]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568
